# World Health Organization Proportion Query #

In [ ]:
!pip install ddf-utils
!pip install tqdm

In [ ]:
import pandas as pd
import numpy as np
import requests
from tqdm import tqdm
import json

In [ ]:
def get_who_data(proportion):
    p = requests.get(f"https://ghoapi.azureedge.net/api/Indicator?$filter=contains(IndicatorName,'{proportion}')")
    proportion_ind = p.json()['value']
    indicator_df = pd.DataFrame(proportion_ind)
    
    c = requests.get("https://ghoapi.azureedge.net/api/DIMENSION/COUNTRY/DimensionValues")
    countries = c.json()['value']
    country_df = pd.DataFrame(countries)
    country_df = country_df.rename(columns={'Code': 'SpatialDim'})

    all_proportion = {}

    for code in tqdm(indicator_df['IndicatorCode'], desc=f"Downloading {proportion} indicators"):
        url = f"https://ghoapi.azureedge.net/api/{code}"
        r = requests.get(url, timeout=30)

        if r.status_code == 200:
            res_df = pd.DataFrame(r.json()['value'])
            res_df['IndicatorCode'] = code
            merged = res_df.merge(indicator_df[['IndicatorCode', 'IndicatorName']], on='IndicatorCode', how='left')
            all_proportion[code] = merged

        else:
            print(f"{code}: HTTP {r.status_code}")

    non_empty = [df for df in all_proportion.values() if not df.empty]

    all_df = pd.concat(non_empty, ignore_index=True)

    all_df.to_csv(f"who_{proportion}_data.tsv", sep='\t', index=False)

    return all_df

incidence = get_who_data("incidence")
incidence.head()

,IndicatorCode,IndicatorName,Language
0,LEPTO_MORBIDITY,Estimated leptospirosis incidence (per 100 000...,EN
1,MALARIA005,Malaria incidence per 100 000 population,EN
2,MDG_0000000020,Incidence of tuberculosis (per 100 000 populat...,EN
3,NTD_3,Annual incidence of dracunculiasis cases,EN
4,SA_0000001463_ARCHIVED,"Alcoholic psychosis, incidence, per 100,000",EN
